In [1]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

In [2]:
#Parameters
file = "../tables/2021AP.xlsx"
next_week = 10
#Read in df
df = pd.read_excel(file)
df

,Team,Week,Rank,W,L,Winning Percentage,Opp. Rank,Opp. P5,Home,Result,Points Scored,Points Against,Margin,Next Week Rank,Movement
0,Alabama,1,1,0,0,1,14,1,0,W,44,13,31,1,0
1,Oklahoma,1,2,0,0,1,26,0,1,W,40,35,5,4,-2
2,Clemson,1,3,0,0,1,5,1,0,L,3,10,-7,6,-3
3,Ohio State,1,4,0,0,1,26,1,0,W,45,31,14,3,1
4,Georgia,1,5,0,0,1,3,1,0,W,10,3,7,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,Penn State,10,22,5,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
247,SMU,10,23,7,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
248,Louisiana-Lafayette,10,24,7,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
249,Fresno State,10,25,7,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
#Drop footer
df.drop(index = (df.index.stop - 1), inplace = True)

#Drop null columns (teams with byes)
df.dropna(inplace = True)

#Set week
for index, row in df.iterrows():
    if row['Week'] >= next_week:
        df.drop(index = index, inplace = True)

In [4]:
df

,Team,Week,Rank,W,L,Winning Percentage,Opp. Rank,Opp. P5,Home,Result,Points Scored,Points Against,Margin,Next Week Rank,Movement
0,Alabama,1,1,0,0,1,14,1,0,W,44,13,31,1,0
1,Oklahoma,1,2,0,0,1,26,0,1,W,40,35,5,4,-2
2,Clemson,1,3,0,0,1,5,1,0,L,3,10,-7,6,-3
3,Ohio State,1,4,0,0,1,26,1,0,W,45,31,14,3,1
4,Georgia,1,5,0,0,1,3,1,0,W,10,3,7,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,Penn State,9,20,5,2,0.714286,5,1,0,L,24,33,-9,22,-2
220,San Diego State,9,21,7,0,1,26,0,1,L,20,30,-10,26,-5
221,Iowa State,9,22,5,2,0.714286,26,1,0,L,31,38,-7,26,-4
223,Coastal Carolina,9,24,6,1,0.857143,26,0,1,W,35,28,7,21,3


In [5]:
#Make W/L binary values
for index, row in df.iterrows():
    if row['Result'] == 'W':
        df.at[index, 'Result'] = 1
    else: 
        df.at[index, 'Result'] = 0

In [6]:
df = df.drop(columns = ["Movement", "Team"])

In [7]:
for name, values in df.iteritems():
    df[name] = pd.to_numeric(values)
    
df.dtypes

Week                    int64
Rank                    int64
W                       int64
L                       int64
Winning Percentage    float64
Opp. Rank               int64
Opp. P5                 int64
Home                    int64
Result                  int64
Points Scored           int64
Points Against          int64
Margin                  int64
Next Week Rank          int64
dtype: object

In [8]:
#Group rankings to categories
for index, row in df.iterrows():
    if row['Rank'] <= 5:
        df.at[index, 'Rank'] = 0
    elif row['Rank'] <= 10:
        df.at[index, 'Rank'] = 1
    elif row['Rank'] <= 15:
        df.at[index, 'Rank'] = 2
    elif row['Rank'] <= 20:
        df.at[index, 'Rank'] = 3
    elif row['Rank'] <= 25:
        df.at[index, 'Rank'] = 4
    else:
        df.at[index, 'Rank'] = 5
        
for index, row in df.iterrows():
    if row['Next Week Rank'] <= 5:
        df.at[index, 'Next Week Rank'] = 0
    elif row['Next Week Rank'] <= 10:
        df.at[index, 'Next Week Rank'] = 1
    elif row['Next Week Rank'] <= 15:
        df.at[index, 'Next Week Rank'] = 2
    elif row['Next Week Rank'] <= 20:
        df.at[index, 'Next Week Rank'] = 3
    elif row['Next Week Rank'] <= 25:
        df.at[index, 'Next Week Rank'] = 4
    else:
        df.at[index, 'Next Week Rank'] = 5

In [9]:
X = df.drop(columns = ['Next Week Rank'])
y = df["Next Week Rank"]

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 8)
X_train.shape

(151, 12)

In [11]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [12]:
#Decision Tree classifier
from sklearn import tree
dtc = tree.DecisionTreeClassifier(random_state = 8)
dtc.fit(X_train, y_train)
dtc.score(X_test, y_test)

0.6274509803921569

In [13]:
#Scaled Decision Tree classifier
from sklearn import tree
scaled_dtc = tree.DecisionTreeClassifier(random_state = 8)
scaled_dtc.fit(X_train_scaled, y_train)
scaled_dtc.score(X_test_scaled, y_test)

0.6274509803921569

In [14]:
#Random Forest classifier
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state = 8)
rfc.fit(X_train, y_train)
rfc.score(X_test, y_test)

0.6078431372549019

In [15]:
#Scaled Random Forest classifier
from sklearn.ensemble import RandomForestClassifier
scaled_rfc = RandomForestClassifier(random_state = 8)
scaled_rfc.fit(X_train_scaled, y_train)
scaled_rfc.score(X_test_scaled, y_test)

0.6274509803921569

In [16]:
#Support Vector Machine classifier
from sklearn.svm import SVC
svm = SVC(random_state = 8)
svm.fit(X_train, y_train)
svm.score(X_test, y_test)

0.27450980392156865

In [17]:
#Scaled SVMC
svm = SVC(random_state = 8)
svm.fit(X_train_scaled, y_train)
svm.score(X_test_scaled, y_test)

0.37254901960784315